# Advent of Code 2018

See [here](http://adventofcode.com/2018/).

## Preparation

Imports and utility functions that might or might not prove useful down the line.

In [1]:
# Python 3.x
import re
import numpy as np
import math
import random
import urllib.request
import reprlib
import operator
import string
import hashlib
import json

from collections import Counter, defaultdict, namedtuple, deque
from functools   import lru_cache, reduce
from itertools   import (permutations, combinations, chain, cycle, tee,
                        product, islice, count, repeat, filterfalse, accumulate)
from heapq       import heappop, heappush
from enum        import Enum

# sometimes a graph helps
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (8,8)

def Input(day,strip=True):
    "Open this day's input file."
    
    filename = 'input/input{}.txt'.format(day)
    try:
        with open(filename, 'r') as f:
            text = f.read()
            if strip:
                text = text.strip()
        return text
    except FileNotFoundError:
        url = 'http://adventofcode.com/2018/day/{}/input'.format(day)
        print('input file not found. opening browser...')
        print('please save the file as "input<#day>.txt in your input folder.')
        import webbrowser
        webbrowser.open(url)

cat = ''.join
def first(iterable, default=None): return next(iter(iterable), default)
def nth(iterable, n, default=None): return next(islice(iterable, n, None), default)
def fs(*items): return frozenset(items)

def window(seq, n=2):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result

def ilen(iterator): return sum(1 for _ in iterator)

def ints(text,typ=int):
    return list(map(typ,re.compile(r'[-+]?\d*[.]?\d+').findall(text)))

def shift(it, n):
    return it[n:] + it[:n]

def rot(mat, N=1, clockwise=True):
    '''rotate 2D matrix'''
    for _ in range(N):
        if clockwise:
            mat = list(zip(*mat[::-1]))
        else:
            mat = list(zip(*mat[::-1]))[::-1]
    return mat

def locate2D(m, val):
    '''locate value in 2D list'''
    for i, line in enumerate(m):
        j=-1
        try:
            j = line.index(val)
        except ValueError:
            continue
        break
    else:
        i = -1
    return (i,j)

def dist_L1(p1,p2=None):
    if p2 == None:
        p2 = repeat(0)
    return sum(abs(p2_i-p1_i) for p1_i, p2_i in zip(p1,p2))

def dist_L2(p1,p2=None):
    if p2 == None:
        p2 = repeat(0)
    return sum((p2_i-p1_i)*(p2_i-p1_i) for p1_i, p2_i in zip(p1,p2))**.5

def neighbors4(point): 
    "The four neighbors (without diagonals)."
    x, y = point
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1))

def neighbors8(point): 
    "The eight neighbors (with diagonals)."
    x, y = point 
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
            (x+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))

from numbers import Number 
class Vector(object):
    def __init__(self,*args):
        if len(args) == 1:
            if isinstance(args,Number): self.vec = tuple(0 for _ in range(args))
            else: self.vec = tuple(*args)
        else: self.vec = tuple(args)
    def __mul__(self, other):
        if isinstance(other,Number): return Vector(other * x for x in self.vec)
        elif isinstance(other,Vector): return sum(x*y for x,y in zip(self.vec, other.vec))
        raise NotImplemented
    def __add__(self,other):
        return Vector(x+y for x,y in zip(self.vec, other.vec))
    def __sub__(self,other):
        return Vector(x-y for x,y in zip(self.vec, other.vec))
    def __iter__(self):
        return self.vec.__iter__()
    def __len__(self):
        return len(self.vec)
    def __getitem__(self, key):
        return self.vec[key]
    def __repr__(self):
        return 'Vector(' + str(self.vec)[1:-1] + ')'
    def __eq__(self, other):
        return self.vec == other.vec
    def __hash__(self):
        return hash(self.vec)

#display and debug functions
def h1(s):
    upr, brd, lwr = '▁', '█', '▔'
    return upr*(len(s)+4) + '\n'+brd+' ' + s + ' ' + brd +'\n' + lwr*(len(s)+4)

def h2(s, ch='-'):
    return s + '\n' + ch*len(s) + '\n'

h1 = lambda s: h2(s,'=')  #the other h1 is a bitch, apparently.

def print_result(day, part, text):
    print(h1('Day {} part {}: {}'.format(day, part, text)))

def trace1(f):
    "Print a trace of the input and output of a function on one line."
    rep = reprlib.aRepr
    rep.maxother = 85
    def traced_f(*args):
        arg_strs = ', '.join(map(rep.repr, args))
        result = f(*args)
        print('{}({}) = {}'.format(f.__name__, arg_strs, result))
        return result
    return traced_f

## [Day 1: Chronal Calibration](http://adventofcode.com/2018/day/1)

In [17]:
data = Input(1)
frequencies = ints(data)

day1_part1 = sum

def day1_part2(frequencies):
    r = 0
    s = set([r])
    for f in cycle(frequencies):
        r += f
        if r in s:
            return r
        s.add(r)

In [18]:
print_result(1,1,'Frequency is '+str(day1_part1(frequencies)))
print_result(1,2,'first double is ' + str(day1_part2(frequencies)))

Day 1 part 1: Frequency is 439

Day 1 part 2: first double is 124645



## [Day 2: Inventory Management System](http://adventofcode.com/2018/day/2)

In [29]:
data = Input(2).split()

def day2_part1(ids):
    """computing the checksum"""
    data = [Counter(s) for s in ids]
    data = [list(set(v for k,v in s.items() if v == 2 or v == 3)) for s in data if 2 in s.values() or 3 in s.values()]
    data = Counter(sum(data,[]))
    checksum = data[2]*data[3]
    return checksum

test_p1 = ['abcdef', 'bababc', 'abbcde', 'abcccd', 'aabcdd', 'abcdee', 'ababab']
assert day2_part1(test_p1) == 12

def day2_part2(ids):
    """find common letters between matching IDs"""
    for id_1, id_2 in combinations(ids,2):
        common = cat(c_1 for c_1, c_2 in zip(id_1, id_2) if c_1 == c_2)
        if len(common) + 1 == len(id_1):
            return common

test_p2 = ['abcde', 'fghij', 'klmno', 'pqrst', 'fguij', 'axcye', 'wvxyz'] 
assert day2_part2(test_p2) == 'fgij'

In [27]:
print_result(2,1,'checksum is '+str(day2_part1(data)))
print_result(2,2,'common letters for the matching IDs are "'+str(day2_part2(data))+'"')

Day 2 part 1: checksum is 7192

Day 2 part 2: common letters for the matching IDs are "mbruvapghxlzycbhmfqjonsie"



## [Day 3: No Matter How You Slice It](http://adventofcode.com/2018/day/3)


The Elves managed to locate the chimney-squeeze prototype fabric for
Santa's suit (thanks to <span title="WAS IT YOU">someone</span> who
helpfully wrote its box IDs on the wall of the warehouse in the middle
of the night). Unfortunately, anomalies are still affecting them -
nobody can even agree on how to *cut* the fabric.

The whole piece of fabric they're working on is a very large square - at
least `1000` inches on each side.

Each Elf has made a *claim* about which area of fabric would be ideal
for Santa's suit. All claims have an ID and consist of a single
rectangle with edges parallel to the edges of the fabric. Each claim's
rectangle is defined as follows:

  - The number of inches between the left edge of the fabric and the
    left edge of the rectangle.
  - The number of inches between the top edge of the fabric and the top
    edge of the rectangle.
  - The width of the rectangle in inches.
  - The height of the rectangle in inches.

A claim like `#123 @ 3,2: 5x4` means that claim ID `123` specifies a
rectangle `3` inches from the left edge, `2` inches from the top edge,
`5` inches wide, and `4` inches tall. Visually, it claims the square
inches of fabric represented by `#` (and ignores the square inches of
fabric represented by `.`) in the diagram below:

    ...........
    ...........
    ...#####...
    ...#####...
    ...#####...
    ...#####...
    ...........
    ...........
    ...........

The problem is that many of the claims *overlap*, causing two or more
claims to cover part of the same areas. For example, consider the
following claims:

    #1 @ 1,3: 4x4
    #2 @ 3,1: 4x4
    #3 @ 5,5: 2x2

Visually, these claim the following areas:

    ........
    ...2222.
    ...2222.
    .11XX22.
    .11XX22.
    .111133.
    .111133.
    ........

The four square inches marked with `X` are claimed by *both `1` and
`2`*. (Claim `3`, while adjacent to the others, does not overlap either
of them.)

If the Elves all proceed with their own plans, none of them will have
enough fabric. *How many square inches of fabric are within two or more
claims?*

### Part 2
Amidst the chaos, you notice that exactly one claim doesn't overlap by even a single square inch of fabric with any other claim. If you can somehow draw attention to it, maybe the Elves will be able to make Santa's suit after all!

For example, in the claims above, only claim 3 is intact after all claims are made.

What is the ID of the only claim that doesn't overlap?


In [33]:
def d3_parse_claims(data):
    '''created a list of namedtuples claim(id, x, y, w, h) from the input data'''
    claim = namedtuple('claim','id x y w h')
    claims = [ints(d) for d in data.split('\n')]
    claims = [claim(*clm) for clm in claims]
    return claims

def d3_claim_fabric(claims):
    '''create a dict from the claims that has a list of every claim for every square inch'''
    fabric = defaultdict(list)
    for claim in claims:
        for x, y in product(range(claim.x,claim.x+claim.w),
                            range(claim.y,claim.y+claim.h)):
            fabric[(x,y)].append(claim.id)
    return fabric

def day3_part1(claims):
    """find number of square inches with multiple claims"""
    claims = d3_parse_claims(data)
    fabric = d3_claim_fabric(claims)
    return ilen(k for k,v in fabric.items() if len(v) > 1)

def day3_part2(claims):
    '''find only claim that does not share its area with other claims'''
    claims = d3_parse_claims(data)
    fabric = d3_claim_fabric(claims)
    num_overlaps = defaultdict(int)
    for sq_claims in fabric.values():
        for sq_claim in sq_claims:
            num_overlaps[sq_claim] = max(num_overlaps[sq_claim], len(sq_claims))
    min_overlap = min(num_overlaps, key=num_overlaps.get)
    assert num_overlaps[min_overlap] == 1
    return min_overlap

In [34]:
data = Input(3)
print_result(3,1,''+str(day3_part1(data)))
print_result(3,2,''+str(day3_part2(data)))

Day 3 part 1: 113576

Day 3 part 2: 825

